In [4]:
#fatigue prediction in Video
#tou ming xiao guo
#using PIL
#print accuracy per 1 frame
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from PIL import ImageFont, ImageDraw, Image
from keras.preprocessing.image import img_to_array
import numpy as np
import os
import cv2

fatigueDegree = {0 : 'DROS 1',
                 1 : 'DROS 2',
                 2 : 'DROS 3'}

path = 'D:\\'
videoName = 'depthCam_Driver_1_13_6_8_19'

inputVideoPath = os.path.join(path,videoName)+'.avi'
outputVideoPath = path+'\\result_16_'+videoName+'.avi'

modelPath = 'D:\\models\\inceptionv3_lstm_timeStep16_test.h5'

detector = MTCNN()
model = load_model(modelPath)

capture = cv2.VideoCapture(inputVideoPath)

fps = int(round(capture.get(cv2.CAP_PROP_FPS)))


width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(outputVideoPath,fourcc,fps,(width,height))

flag = True
while flag:
    imageNum = 0
    for i in range(fps):
        ret,frame = capture.read()
        if not ret:
            flag = False
            break

        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        frame_copy = frame.copy()
        
        face = detector.detect_faces(frame)
        
        if not face:
            continue

        boxList = np.zeros(len(face))
        for i in range(len(face)):
            box = face[i]['box']
            boxList[i] = box[2]*box[3]

        maxBoxIndex = np.argmax(boxList)
        maxBox = face[maxBoxIndex]['box']

        cropped = frame[maxBox[1]:maxBox[1]+maxBox[3],maxBox[0]:maxBox[0]+maxBox[2]]
        
        x = cv2.resize(cropped,(200,150),interpolation = cv2.INTER_AREA)
        x = img_to_array(x)
        x = np.expand_dims(x, axis=0)
        x /= 255

        f_degree = model.predict(x)
        
        mask_width = 150
        _,img_width = frame.shape[:2]
        box_height = 100
        
        font = ImageFont.truetype('arial.ttf',15)
        font_1 = ImageFont.truetype('arial.ttf',20)

        cv2.rectangle(frame,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(67,116,217),1)

        points = np.array([[box[0]+box[2]+5,box[1]],
                               [box[0]+box[2]+5+mask_width,box[1]],
                               [box[0]+box[2]+5+mask_width,box[1]+box_height],
                               [box[0]+box[2]+5,box[1]+box_height]],np.int32)

        sub_img = frame_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]
        roi = frame[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]
        if (box[0]+box[2]+5+mask_width) > img_width:
            points = np.array([[box[0]+box[2]+5,box[1]],
                               [box[0]+box[2]+5+img_width,box[1]],
                               [box[0]+box[2]+5+img_width,box[1]+box_height],
                               [box[0]+box[2]+5,box[1]+box_height]],np.int32)

            sub_img = frame_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+img_width]
            roi = frame[box[1]:box[1]+box_height,box[0]+box[2]+5:img_width]

        background = cv2.fillConvexPoly(frame,points,(76,76,76))

        dst = cv2.addWeighted(roi,0.5,sub_img,0.5,0)

        background[points[0][1]:points[2][1],points[0][0]:points[2][0]] = dst
        
        background = Image.fromarray(background)
        draw = ImageDraw.Draw(background)
        
        imageNum += 1            

        draw.text((box[0]+box[2]+7,box[1]), 'DROSLevel: ', font=font_1, fill=(255,255,255))
        
        maxIndex = np.argmax(f_degree)
        distance = 20
        
        if imageNum==0:
            draw.text((box[0]+box[2]+9,box[1]+(box_height//2)), 'Recognizing...', font=font_1, fill=(255,228,0))    
        else:
            for i in range(len(fatigueDegree)):
                if i == maxIndex:
                    draw.text((box[0]+box[2]+7, box[1]+distance+15), fatigueDegree[i]+f':{f_degree[0][i] : .2%}', font=font, fill=(255, 228, 0))
                else:    
                    draw.text((box[0]+box[2]+7, box[1]+distance+15), fatigueDegree[i]+f':{f_degree[0][i] : .2%}', font=font, fill=(255, 255, 255))
                distance += 21

        background = cv2.cvtColor(np.array(background), cv2.COLOR_RGB2BGR)
        cv2.imshow('result',background)
        out.write(background)
    
    key = cv2.waitKey(3)
    if key == 27:
        break
        print('ese break...')
        
capture.release()
out.release()
cv2.destroyAllWindows()

In [2]:
import numpy as np
import os
import cv2

path = 'D:\\'
videoName = 'rightCam_Driver_1_13_6_8_19'

inputVideoPath = os.path.join(path,videoName)+'.avi'
outputVideoPath = path+'\\result_16_'+videoName+'.avi'

capture = cv2.VideoCapture(inputVideoPath)

fps = int(round(capture.get(cv2.CAP_PROP_FPS)))


width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(outputVideoPath,fourcc,fps,(width,height))

flag = True
while flag:
    for i in range(fps):
        ret,frame = capture.read()
        if not ret:
            flag = False
            break

        frame = cv2.applyColorMap(frame, cv2.COLORMAP_JET)
        cv2.imshow('result',frame)
        out.write(frame)
    
    key = cv2.waitKey(3)
    if key == 27:
        break
        print('ese break...')
        
capture.release()
out.release()
cv2.destroyAllWindows()

In [1]:
#print accuracy per 20 frame

from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from PIL import ImageFont, ImageDraw, Image
from keras.preprocessing.image import img_to_array
import numpy as np
import os
import cv2

fatigueDegree = {0 : 'DROS 1',
                 1 : 'DROS 2',
                 2 : 'DROS 3'}


path = 'C:\\Users\\CHENTIEJUN\\Desktop\\testvidel'
videoName = '12'

inputVideoPath = os.path.join(path,videoName)+'.avi'
outputVideoPath = path+'\\result_16_'+videoName+'.avi'

modelPath = 'D:\\models\\inceptionv3_lstm_timeStep16.h5'

detector = MTCNN()
model = load_model(modelPath)

capture = cv2.VideoCapture(inputVideoPath)

fps = int(round(capture.get(cv2.CAP_PROP_FPS)))


width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(outputVideoPath,fourcc,fps,(width,height))

averageAccuracy = np.array([[0,0,0]])
flag = True
while flag:
    totalAccuracy = np.zeros((1,3))
    imageNum = 0
    for i in range(fps):
        ret,frame = capture.read()
        if not ret:
            flag = False
            break

        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        frame_copy = frame.copy()
        
        face = detector.detect_faces(frame)
        
        if not face:
            continue

        boxList = np.zeros(len(face))
        for i in range(len(face)):
            box = face[i]['box']
            boxList[i] = box[2]*box[3]

        maxBoxIndex = np.argmax(boxList)
        maxBox = face[maxBoxIndex]['box']

        cropped = frame[maxBox[1]:maxBox[1]+maxBox[3],maxBox[0]:maxBox[0]+maxBox[2]]
        
        x = cv2.resize(cropped,(200,150),interpolation = cv2.INTER_AREA)
        x = img_to_array(x)
        x = np.expand_dims(x, axis=0)
        x /= 255

        custom = model.predict(x)
        
        mask_width = 150
        _,img_width = frame.shape[:2]
        box_height = 100
        
        font = ImageFont.truetype('arial.ttf',15)
        font_1 = ImageFont.truetype('arial.ttf',20)

        cv2.rectangle(frame,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(67,116,217),1)

        points = np.array([[box[0]+box[2]+5,box[1]],
                               [box[0]+box[2]+5+mask_width,box[1]],
                               [box[0]+box[2]+5+mask_width,box[1]+box_height],
                               [box[0]+box[2]+5,box[1]+box_height]],np.int32)

        sub_img = frame_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]
        roi = frame[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]
        if (box[0]+box[2]+5+mask_width) > img_width:
            points = np.array([[box[0]+box[2]+5,box[1]],
                               [box[0]+box[2]+5+img_width,box[1]],
                               [box[0]+box[2]+5+img_width,box[1]+box_height],
                               [box[0]+box[2]+5,box[1]+box_height]],np.int32)

            sub_img = frame_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+img_width]
            roi = frame[box[1]:box[1]+box_height,box[0]+box[2]+5:img_width]

        background = cv2.fillConvexPoly(frame,points,(76,76,76))

        dst = cv2.addWeighted(roi,0.5,sub_img,0.5,0)

        background[points[0][1]:points[2][1],points[0][0]:points[2][0]] = dst
        
        background = Image.fromarray(background)
        draw = ImageDraw.Draw(background)

        totalAccuracy += custom
        imageNum += 1
        
        if (imageNum != 0 and imageNum % 20 == 0):
            averageAccuracy = totalAccuracy / imageNum

        draw.text((box[0]+box[2]+7,box[1]), 'DROSLevel:', font=font_1, fill=(255,255,255))
        
        maxIndex = np.argmax(averageAccuracy)
        distance = 20
        
        if np.sum(averageAccuracy)==0:
            draw.text((box[0]+box[2]+9,box[1]+(box_height//2)), 'Recognizing...', font=font_1, fill=(255,228,0))    
        else:
            for i in range(len(fatigueDegree)):
                if i == maxIndex:
                    draw.text((box[0]+box[2]+7,box[1]+distance+15), fatigueDegree[i]+f':{averageAccuracy[0][i] : .2%}', font=font, fill=(255,228,0))
                else:    
                    draw.text((box[0]+box[2]+7,box[1]+distance+15), fatigueDegree[i]+f':{averageAccuracy[0][i] : .2%}', font=font, fill=(255,255,255))
                distance += 21

        background = cv2.cvtColor(np.array(background),cv2.COLOR_RGB2BGR)
        cv2.imshow('result',background)
        out.write(background)
    
    key = cv2.waitKey(3)
    if key == 27:
        break
        print('ese break...')
        
capture.release()
out.release()
cv2.destroyAllWindows()

Using TensorFlow backend.
